# Notebook pro trénink s destilací nad datasetem CIFAR100
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR100, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

## Import knihoven a definice metod

In [1]:
from transformers import Trainer, EarlyStoppingCallback, AutoModelForImageClassification
from torch.utils.data import ConcatDataset, DataLoader
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
dataset_part = base.get_dataset_part()
DATASET = "cifar10"

Inicializovaný MobileNetV2.

In [3]:
base.reset_seed()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [5]:
transform = base.base_transforms()

train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform)
test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)


In [6]:
augment_transform = base.aug_transforms()

train_aug = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)

In [7]:
train_part_cpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=True, batch=1, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
train_part_gpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=True, batch=1, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)

In [8]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
print(len(train_aug))
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

28176


### Standardní trénink náhodně inicializovaného modelu. 

In [11]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-basetrain", logging_dir=f"~/logs/{DATASET}/vit-basetrain", lr=0.0005, weight_decay=0.008, adam_beta1=.95, epochs=30, batch_size=64)
model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=10, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.452100,0.298889,0.903100,0.906501,0.902874,0.903310
2,0.226200,0.268658,0.906500,0.912503,0.906892,0.905723
3,0.166300,0.232208,0.925200,0.927676,0.925357,0.925583
4,0.132700,0.216103,0.931500,0.933343,0.931734,0.931739
5,0.110600,0.231508,0.929300,0.931600,0.929432,0.929507
6,0.085800,0.222461,0.933200,0.935557,0.933829,0.933379
7,0.072600,0.237990,0.937200,0.937652,0.937521,0.937338
8,0.062100,0.208141,0.943300,0.944872,0.943425,0.943536
9,0.052900,0.195802,0.948100,0.948556,0.948412,0.948088
10,0.050600,0.197401,0.947000,0.947961,0.947230,0.947264


In [ ]:
model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.9488574266433716,
 'eval_accuracy': 0.8366,
 'eval_precision': 0.8385374944070844,
 'eval_recall': 0.8366,
 'eval_f1': 0.8365053909234638,
 'eval_runtime': 28.9148,
 'eval_samples_per_second': 345.844,
 'eval_steps_per_second': 2.732,
 'epoch': 30.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-basetrain.pth")

In [ ]:
base.count_parameters(model)

In [ ]:
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

In [ ]:
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-distilltrain", logging_dir=f"~/logs/{DATASET}/vit-distilltrain", remove_unused_columns=False, epochs=30, lr=0.00047, weight_decay=0, adam_beta1=.9, lambda_param=1, temp=6)
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=10, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.298900,0.831203,0.683800,0.731836,0.683800,0.685063
2,0.600000,0.662224,0.729600,0.753612,0.729600,0.730670
3,0.453300,0.622127,0.755500,0.776564,0.755500,0.753489
4,0.368500,0.563021,0.770700,0.789720,0.770700,0.772211
5,0.310300,0.546491,0.779300,0.799878,0.779300,0.779068
6,0.262700,0.532122,0.782600,0.800717,0.782600,0.784513
7,0.232000,0.495274,0.797000,0.812961,0.797000,0.799272
8,0.203200,0.501735,0.795400,0.811297,0.795400,0.797779
9,0.185400,0.486403,0.797000,0.811036,0.797000,0.797992
10,0.164500,0.481768,0.797900,0.809001,0.797900,0.798922


TrainOutput(global_step=9390, training_loss=0.18513636878504158, metrics={'train_runtime': 4341.4904, 'train_samples_per_second': 276.403, 'train_steps_per_second': 2.163, 'total_flos': 5.5315759693824e+18, 'train_loss': 0.18513636878504158, 'epoch': 30.0})

In [ ]:
student_model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.2674492597579956,
 'eval_accuracy': 0.8355,
 'eval_precision': 0.8430531401010704,
 'eval_recall': 0.8355,
 'eval_f1': 0.8366243954260613,
 'eval_runtime': 13.4606,
 'eval_samples_per_second': 742.908,
 'eval_steps_per_second': 5.869,
 'epoch': 30.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-distilltrain.pth")

In [ ]:
base.count_parameters(student_model)

In [ ]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

In [ ]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-base-aug", logging_dir=f"~/logs/{DATASET}/vit-base-aug", lr=0.0005, weight_decay=0.008, adam_beta1=.95, epochs=30)
model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=10, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.470800,1.027773,0.700900,0.722194,0.700900,0.697797
2,0.688500,0.920618,0.733700,0.761423,0.733700,0.732250
3,0.484900,0.851626,0.761700,0.779286,0.761700,0.761634
4,0.361100,0.848293,0.779900,0.792589,0.779900,0.780535
5,0.278600,0.856959,0.779300,0.789564,0.779300,0.779449
6,0.217200,0.852996,0.781800,0.792076,0.781800,0.781594
7,0.175200,0.938379,0.773600,0.785053,0.773600,0.773906
8,0.144200,0.889523,0.788300,0.799997,0.788300,0.789484
9,0.121700,0.916836,0.787400,0.802762,0.787400,0.787534
10,0.098300,0.924253,0.797100,0.803304,0.797100,0.796267


TrainOutput(global_step=15450, training_loss=0.15049648686664777, metrics={'train_runtime': 4609.6179, 'train_samples_per_second': 428.964, 'train_steps_per_second': 3.352, 'total_flos': 9.114930882348319e+18, 'train_loss': 0.15049648686664777, 'epoch': 30.0})

In [ ]:
model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [ ]:
trainer.evaluate(test)

{'eval_loss': 1.0624759197235107,
 'eval_accuracy': 0.8323,
 'eval_precision': 0.8333352414020642,
 'eval_recall': 0.8323,
 'eval_f1': 0.8319760201582059,
 'eval_runtime': 12.1123,
 'eval_samples_per_second': 825.61,
 'eval_steps_per_second': 6.522,
 'epoch': 30.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-basetrain-aug.pth")

In [ ]:
base.count_parameters(model)

In [ ]:
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

In [ ]:
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

Destilace

In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-distill-aug", logging_dir=f"~/logs/{DATASET}/vit-distill-aug", remove_unused_columns=False, epochs=30, lr=0.00047, weight_decay=0, adam_beta1=.9, lambda_param=1, temp=6)
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=10, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.090500,0.772572,0.702900,0.723211,0.702900,0.699146
2,0.555700,0.608115,0.756500,0.774005,0.756500,0.757947
3,0.429300,0.579290,0.772000,0.787107,0.772000,0.773073
4,0.360500,0.545981,0.781500,0.794083,0.781500,0.782081
5,0.307500,0.525294,0.780800,0.797996,0.780800,0.781727
6,0.269300,0.518046,0.787800,0.802332,0.787800,0.788994
7,0.241000,0.492204,0.800100,0.810344,0.800100,0.800842
8,0.217300,0.490440,0.793400,0.809793,0.793400,0.794289
9,0.199000,0.473009,0.800400,0.813401,0.800400,0.801665
10,0.181600,0.476089,0.801100,0.814297,0.801100,0.803030


TrainOutput(global_step=15450, training_loss=0.18576704130203592, metrics={'train_runtime': 7036.1157, 'train_samples_per_second': 281.03, 'train_steps_per_second': 2.196, 'total_flos': 9.114930882348319e+18, 'train_loss': 0.18576704130203592, 'epoch': 30.0})

In [ ]:
student_model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.2835237979888916,
 'eval_accuracy': 0.8274,
 'eval_precision': 0.8334670865429635,
 'eval_recall': 0.8273999999999999,
 'eval_f1': 0.8279914738310015,
 'eval_runtime': 12.9351,
 'eval_samples_per_second': 773.092,
 'eval_steps_per_second': 6.107,
 'epoch': 30.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-distilltrain_aug.pth")

In [ ]:
base.count_parameters(student_model)

In [ ]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

In [ ]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())